In [1]:
import json, pprint, requests, textwrap

In [2]:
host = 'http://master:8998'
data = {'kind': 'pyspark'}
headers = {'Content-Type': 'application/json'}

## Set spark session

In [3]:
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [4]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 0,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ',
  '\nstderr: ',
  '21/05/07 16:06:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
  'log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).',
  'log4j:WARN Please initialize the log4j system properly.',
  'log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.']}

## Sample code 1

In [5]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{'id': 0,
 'code': '1 + 1',
 'state': 'waiting',
 'output': None,
 'progress': 0.0,
 'started': 0,
 'completed': 0}

## Sample code 2

In [7]:
data = {
  'code': textwrap.dedent("""
import random
NUM_SAMPLES = 100000
def sample(p):
    x, y = random.random(), random.random()
    return 1 if x*x + y*y < 1 else 0

count = sc.parallelize(range(0, NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)

print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))
    """)
}

r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{'code': '\n'
         'import random\n'
         'NUM_SAMPLES = 100000\n'
         'def sample(p):\n'
         '    x, y = random.random(), random.random()\n'
         '    return 1 if x*x + y*y < 1 else 0\n'
         '\n'
         'count = sc.parallelize(range(0, '
         'NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)\n'
         '\n'
         'print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))\n',
 'completed': 0,
 'id': 2,
 'output': None,
 'progress': 0.0,
 'started': 1620371271906,
 'state': 'running'}


In [8]:
statement_url = host + r.headers['location']
r = requests.get(statement_url, headers=headers)
pprint.pprint(r.json())

{'code': '\n'
         'import random\n'
         'NUM_SAMPLES = 100000\n'
         'def sample(p):\n'
         '    x, y = random.random(), random.random()\n'
         '    return 1 if x*x + y*y < 1 else 0\n'
         '\n'
         'count = sc.parallelize(range(0, '
         'NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)\n'
         '\n'
         'print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))\n',
 'completed': 1620371274510,
 'id': 2,
 'output': {'data': {'text/plain': 'Pi is roughly 3.138240'},
            'execution_count': 2,
            'status': 'ok'},
 'progress': 1.0,
 'started': 1620371271906,
 'state': 'available'}


## Delete spark session

In [17]:
session_url = 'http://master:8998/sessions/0'
requests.delete(session_url, headers=headers)

<Response [200]>